<a href="https://colab.research.google.com/github/prayogalazmi/torrent-video-downloader-compressor/blob/main/Torrent_Video_Downloader_%26_Compressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📡 Torrent Video Downloader & Compressor
> **High-Speed Torrent to Direct Link | Auto-Save to Google Drive**

---

### ⚡ **Quick Start Guide**

*   **Instant Mode (Direct Download or Audio Extract):**
    If you only want to generate a direct download link or extract audio (MP3), you can **start immediately** without enabling the GPU. It's fast and efficient!

*   **Compression Mode (Video Re-encoding):**
    If you want to compress your videos (720p, 480p, etc.), it is **HIGHLY RECOMMENDED** to enable the **T4 GPU** for 10x faster processing.
    
    **How to enable GPU:**
    1. Look at the top right corner of this page.
    2. Click the **inverted white triangle icon** (next to the RAM/Disk status).
    3. Select **"Change runtime type"**.
    4. Under Hardware accelerator, choose **"T4 GPU"** and click **Save**.

*   **Save to Drive Feature:**
    You can choose to save your files permanently to your **Google Drive** by checking the `save_to_drive` option in Step 1. If unchecked, your files will be stored in the temporary Colab session and deleted once the session ends.

---

## 📖 How to Use
1. **Step 1:** Run "System Preparation" and authorize Google Drive (if enabled).
2. **Step 2:** Paste your **Magnet Link**, run the code, click **"Get File List"**, select your files, and start the download.
3. **Step 3:** Choose your target resolution, run the code to start the encoding process.
4. **Step 4:** Access your files in Google Drive or download them manually from the session storage.

---
© Coded by: @Prayogalazmi - https://www.kaggle.com/code/prayogalazmi/

In [ ]:
#@title 🛠️ Step 1: System Preparation { display-mode: "form" }
#@markdown Do you want the resulting file saved to drive?
from google.colab import drive
import os, warnings

# --- Form Input ---
save_to_drive = False #@param {type:"boolean"}
# ------------------

warnings.filterwarnings("ignore", category=DeprecationWarning)
print("⏳ Installing tools... please wait.")
!pip install libtorrent -q

import libtorrent as lt
import glob, shutil

# Paths configuration
save_path = "/content/temp_cache"
if save_to_drive:
    drive.mount('/content/drive')
    output_folder = "/content/drive/MyDrive/Colab_Downloads"
    print(f"📂 Videos will be saved to: Google Drive (Colab_Downloads folder)")
else:
    output_folder = "/content/videos"
    print(f"📂 Videos will be saved to: Colab Session Storage (Temporary)")

for folder in [save_path, output_folder]:
    if not os.path.exists(folder): os.makedirs(folder)

print("\n🚀 Setup complete!")


In [ ]:
#@title 📥 Step 2: Select and Download { display-mode: "form" }
magnet_link = "" #@param {type:"string"}
#@markdown ---
#@markdown *Paste the Magnet Link above, then run this cell to see the list of files.*

import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import sys

downloaded_files = []
ses = None
handle = None

def get_torrent_info():
    global ses, handle
    if not magnet_link or magnet_link == "input target magnet link":
        print("❌ Error: Please enter a magnet link first!")
        return

    print("🔍 Connecting to peers... Please wait for metadata.")

    # Inisialisasi Session
    ses = lt.session()
    ses.listen_on(6881, 6891)
    params = {"save_path": save_path}
    handle = lt.add_magnet_uri(ses, magnet_link, params)

    # Tunggu Metadata
    timeout = 0
    while not handle.has_metadata():
        time.sleep(1)
        timeout += 1
        if timeout > 60:
            print("⌛ Metadata timeout. Check your link.")
            return

    info = handle.get_torrent_info()
    file_list = [f.path for f in info.files()]

    # UI Elements
    file_selector = widgets.SelectMultiple(
        options=file_list,
        description='Select Files:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='90%', height='200px')
    )

    download_button = widgets.Button(
        description='Start Download Selected',
        button_style='success',
        icon='download',
        layout=widgets.Layout(width='250px')
    )

    output_area = widgets.Output()

    def start_selected_download(b):
        global downloaded_files
        with output_area:
            if not file_selector.value:
                clear_output()
                print("❌ Error: Please select at least one file!")
                return

            selected_files = list(file_selector.value)
            downloaded_files = selected_files

            # Prioritaskan file pilihan (0: skip, 4: download)
            file_priorities = [0] * info.num_files()
            for i, f in enumerate(info.files()):
                if f.path in selected_files:
                    file_priorities[i] = 4

            handle.prioritize_files(file_priorities)

            # Progress Loop
            while handle.status().state != lt.torrent_status.seeding:
                s = handle.status()

                # Trick clear_output(wait=True) agar satu baris dan tidak kedap-kedip
                clear_output(wait=True)
                print(f"🚀 Download started for {len(selected_files)} file(s)...")
                print(f"📊 Progress: {s.progress * 100:.2f}% | ⬇️ Speed: {s.download_rate / 1048576:.2f} MB/s")

                if s.progress >= 1.0 or s.is_seeding:
                    break
                time.sleep(1)

            print("\n✅ Success: Selected files downloaded!")

    download_button.on_click(start_selected_download)

    # Tampilkan UI
    clear_output()
    print(f"✅ Metadata Found: {info.name()}")
    print("💡 Hold Ctrl (Win) or Cmd (Mac) to select multiple files.")
    display(file_selector, download_button, output_area)

# Jalankan fungsi utama
get_torrent_info()

In [ ]:
#@title 🎬 Step 3: Compression or Direct Download { display-mode: "form" }
target_res = "Direct Download" #@param ["Direct Download", "720", "480", "360", "audio"]
#@markdown ---
#@markdown *Select the resolution above and then run this cell to start the process.*

import subprocess
import re
import os
import sys
from IPython.display import clear_output

def run_conversion():
    global downloaded_files
    if not downloaded_files:
        print("⚠️ No record of downloaded files found! Please finish Step 2 first.")
        return

    # Silent GPU Check for Colab T4
    try:
        gpu_check = subprocess.run(['nvidia-smi'], capture_output=True).returncode == 0
    except:
        gpu_check = False

    mode_text = "GPU Acceleration" if gpu_check and target_res != 'Direct Download' else "Standard/Direct"
    print(f"🚀 Mode: {mode_text}")

    for file_path_name in downloaded_files:
        input_video = os.path.join(save_path, file_path_name)
        if not os.path.exists(input_video):
            continue

        file_name = os.path.basename(input_video)
        ext = ".mp3" if target_res == 'audio' else ".mp4"
        output_final = os.path.join(output_folder, f"RESULT_{target_res}_{file_name}{ext}")

        # FFmpeg Logic Parameters
        bitrate_limits = {'720': '2.5M', '480': '1.2M', '360': '700k'}
        max_b = bitrate_limits.get(target_res, '1M')

        if target_res == 'Direct Download':
            ffmpeg_params = "-c copy"
            hw_accel = ""
        elif target_res == 'audio':
            ffmpeg_params = "-vn -c:a libmp3lame -q:a 4"
            hw_accel = ""
        elif gpu_check:
            ffmpeg_params = f"-vf scale_cuda=-2:{target_res} -c:v h264_nvenc -preset p4 -rc vbr -cq 28 -maxrate {max_b} -bufsize 2M -c:a copy"
            hw_accel = "-hwaccel cuda -hwaccel_output_format cuda"
        else:
            ffmpeg_params = f"-vf scale=-2:{target_res} -c:v libx264 -crf 24 -preset faster -maxrate {max_b} -bufsize 2M -c:a copy"
            hw_accel = ""

        # Command FFmpeg (Redirect stderr to stdout agar bisa dibaca)
        cmd = f"ffmpeg {hw_accel} -i \"{input_video}\" {ffmpeg_params} -y \"{output_final}\" 2>&1"

        # Eksekusi dengan Subprocess
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

        for line in process.stdout:
            # Cari baris yang mengandung informasi frame atau size (indikator progres)
            if "frame=" in line or "size=" in line or "time=" in line:
                # Bersihkan area output dan cetak status terbaru
                clear_output(wait=True)
                print(f"🚀 Mode: {mode_text}")
                print(f"⚙️ Now Processing: {file_name}")
                print(f"📊 Status: {line.strip()}")
                sys.stdout.flush() # Paksa Colab nampilin teksnya sekarang

        process.wait()
        if process.returncode == 0:
            print(f"\n✅ Finished: {file_name}")
        else:
            print(f"\n❌ Error on: {file_name}")

    print("\n🎉 All tasks are complete! Your files are in Google Drive (if enabled).")

# Jalankan fungsi
run_conversion()

In [ ]:
#@title 🏁 Step 4: Access Your Files & Cleanup { display-mode: "form" }
import shutil

# --- UI Setup ---
refresh_button = widgets.Button(description='📥 List Processed Files', button_style='success', layout=widgets.Layout(width='250px'))
cleanup_button = widgets.Button(description='🗑️ Delete Cache', button_style='danger', layout=widgets.Layout(width='250px'))
out_files = widgets.Output()

def show_files(b):
    with out_files:
        clear_output()
        files = glob.glob(os.path.join(output_folder, "*"))
        if not files:
            print("📭 No files found.")
            return
        print(f"📂 Files in {output_folder}:")
        for f in files:
            print(f"⬇️ {os.path.basename(f)}")

def clear_cache(b):
    with out_files:
        if os.path.exists(save_path):
            shutil.rmtree(save_path)
            os.makedirs(save_path)
            print("🧹 Cache cleared!")

refresh_button.on_click(show_files)
cleanup_button.on_click(clear_cache)
display(widgets.HBox([refresh_button, cleanup_button]), out_files)
